<a href="https://colab.research.google.com/github/airoprojects/visiope/blob/main/dataloader/load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import torch 
import pickle
from pathlib import Path
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms

import pickle #used to save dataloader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# set up dataset directory path
ckpt = Path('/content/drive/MyDrive/Dataset/')
ckpt.mkdir(exist_ok=True, parents=True)

Mounted at /content/drive


In [ ]:
""" if you already have the daset in your local drive, 
place it in a Dataset directory and unzip it from there """

# unzip dataset
!unzip "/content/drive/MyDrive/Dataset/ai4mars-dataset-merged-0.1.zip" -d "/content/"

In [ ]:
""" else download und unzip the dataset from here """

import gdown

# get url of zipped dataset
url = 'https://drive.google.com/uc?id=1eW9Ah9DDEY02CTHCrRYLGPmiGZvCTKK4'

# set up zip download location and start download
output = '/content/ai4mars-dataset-merged-0.1.zip'
gdown.download(url, output, quiet=False)

# unzip dataset
!unzip "/content/ai4mars-dataset-merged-0.1.zip" -d "/content/"

In [16]:
#Path
images = "ai4mars-dataset-merged-0.1/msl/images"
label_train = "ai4mars-dataset-merged-0.1/msl/labels/train"
label_test_1ag = "ai4mars-dataset-merged-0.1/msl/labels/test/masked-gold-min1-100agree"
label_test_2ag = "ai4mars-dataset-merged-0.1/msl/labels/test/masked-gold-min2-100agree"
label_test_3ag = "ai4mars-dataset-merged-0.1/msl/labels/test/masked-gold-min3-100agree"
edr = images + "/edr"
mxy = images + "/mxy" 
rng = images + "/rng-30m"


edr_files = os.listdir(edr)
label_train_files = os.listdir(label_train)
label_test_files_1 = os.listdir(label_test_1ag)
label_test_files_2 = os.listdir(label_test_2ag)
label_test_files_3 = os.listdir(label_test_3ag)



X = []
y = []

y1 = []
y2 = []
y3 = []

c = 0
size = 224



for label in label_test_files_1:
    path_label = os.path.join(label_test_1ag, label)

    img_arr = cv2.imread(path_label,0) #read image
    #img_arr = cv2.resize(img_arr, dsize=(size, size),interpolation=cv2.INTER_NEAREST) #resize
    y1.append(img_arr)

for label in label_test_files_2:
    path_label = os.path.join(label_test_1ag, label)

    img_arr = cv2.imread(path_label,0) #read image
    #img_arr = cv2.resize(img_arr, dsize=(size, size),interpolation=cv2.INTER_NEAREST) #resize
    y2.append(img_arr)

for label in label_test_files_3:
    path_label = os.path.join(label_test_1ag, label)

    img_arr = cv2.imread(path_label,0) #read image
    #img_arr = cv2.resize(img_arr, dsize=(size, size),interpolation=cv2.INTER_NEAREST) #resize
    y3.append(img_arr)
    

    


for label in label_train_files:


 
    img_name = label[:-4] + ".JPG" #Names of images match names of labels, except for the extension (JPG, png)

    if img_name in edr_files:

        img_path = os.path.join(edr, img_name) #get the full path
        
        # this is ugly, to change!
        img_arr = cv2.imread(img_path) #read image
        #img_arr = cv2.resize(img_arr, dsize=(size, size)) #resize


        label_path = os.path.join(label_train, label)
        lab_arr = cv2.imread(label_path,0) #0 mean read as greyscale image
        #lab_arr = cv2.resize(lab_arr, (size,size), interpolation = cv2.INTER_NEAREST)

        X.append(img_arr)
        print(label_path)
        y.append(lab_arr[np.newaxis, :, :])

        #this control how much images you want
        c+=1
        if c==200: break


X = np.asanyarray(X, dtype= np.float32) / 255
y = np.array(y, dtype= np.int64)







# 0 - soil --> 0 
# 1 - bedrock --> 1
# 2 - sand --> 2
# 3 - big rock --> 3
# 255 -> 4 - NULL (no label)


y[y==255] = 4

print(X.shape, y.shape)
print(y[1])


#from numpy array to torch

Xt = torch.from_numpy(X)
yt = torch.from_numpy(y)




'''
plt.imshow(X[0])
plt.show()
'''



'''
validation set parametrized to decide if you want only 1, 2, oe togheter
parametrized 
'''


(200, 1024, 1024, 3) (200, 1, 1024, 1024)
[[[4 4 4 ... 4 4 4]
  [4 4 4 ... 4 4 4]
  [4 4 4 ... 4 4 4]
  ...
  [4 4 4 ... 4 4 4]
  [4 4 4 ... 4 4 4]
  [4 4 4 ... 4 4 4]]]


'\nvalidation set parametrized to decide if you want only 1, 2, oe togheter\nparametrized \n'

In [17]:
print(Xt.shape)
print(yt.shape)

torch.Size([200, 1024, 1024, 3])
torch.Size([200, 1, 1024, 1024])


In [ ]:

X_rgb = torch.cat((Xt,Xt,Xt), dim=2)
X_rgb.shape


torch.Size([200, 224, 672, 3])

In [ ]:

#This class rappresents the dataset 
class Ai4MarsData(Dataset):
    #X tensor (torch) -> images
    #y tensor (torch) -> labels

    def __init__(self, X, y,transform=None):
        self.X = X.permute(0,3,1,2)
        self.y = y

        self.transform = transform
        
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        image = self.X[index]
        label = self.y[index]

        if self.transform:
            image = self.transform(image)

        return image, label
    

    '''
    TO DO
  
    def resize(self,resize,interp=None):
        if interp:
            k = interp
        else:
            k = cv2.INTER_NEAREST
        for image in self.X:
            image = 
        self.X = cv2.resize(img_arr, dsize=(size, size),interpolation=k)
        self.y = cv2.resize(img_arr, dsize=(size, size),interpolation=k)

    '''
    
    def setPermuteX(self,perm):   
        print(type(self.X)) 
        self.X = self.X.permute(perm[0],perm[1],perm[2],perm[3])
    
    def setPermuteY(self,perm):    
        self.y = self.y.permute(perm[0],perm[1],perm[2],perm[3])

    
    def setDevice(self,device,which):
        if which==0:
            self.X = X.to(device)
        else:
            self.y = y.to(device)

    def convertion(self,what):
        if(what==0):
            self.y = self.y.type(torch.DoubleTensor)
        else:
            self.X = self.X.type(torch.DoubleTensor)


    def resize(self,resize,interp=None):
        '''
        if interp:
            k = interp
        else:
            k = torchvision.transforms.InterpolationMode
        '''
        transform = transforms.Resize(resize,antialias=True)
        self.X = transform(self.X)
        self.y = transform(self.y)
        
    
    #this function return 3 dataloader (train,test,validation) splitted from self 
    #percentage -> give percentage of train size, the rest of percentage is given divided the residual part
    #sizeBatch -> determine the size of batch
    def splitLoader(self,percentage,sizeBatch):
        dataset = self
        ratio = percentage/100

        #setup variables
        d_size = len(self)
        train_size = int(ratio*d_size)
        test_size = int((d_size - train_size)/2)
        validation_size = test_size

        #split
        train_dataset, test_dataset, validation_dataset = random_split(dataset,[train_size,test_size,validation_size])



       
        

        print(type(train_dataset))

        #create other loaders
        train_loader = DataLoader(train_dataset,batch_size=sizeBatch)
        test_loader = DataLoader(test_dataset,batch_size=sizeBatch)
        validation_loader = DataLoader(validation_dataset,batch_size=sizeBatch)



        return train_loader,test_loader,validation_loader
    


        
    


##TEST -> TO DELETE
#create a dataset


dataset = Ai4MarsData(Xt,yt)
print(dataset.__getitem__(0)[0].size())
dataset2 = Ai4MarsData(Xt,yt)

dataset.resize((64,64))
print(dataset.__getitem__(0)[0].size())
print(dataset.__getitem__(0)[1].size())







In [ ]:
# Save the data loader as a pickle file
with open('/content/drive/MyDrive/Dataset/data_loader.pkl', 'wb') as f:
    pickle.dump(data_dict, f)

In [ ]:
# Save the data loader as a pickle file local
with open('data_loader.pkl', 'wb') as f:
    pickle.dump(data_dict, f)